In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import imp
import sys
from sqlalchemy import create_engine

In [2]:
# Create db connection
r2_func_path = r'C:\Data\James_Work\Staff\Heleen_d_W\ICP_Waters\Upload_Template\useful_resa2_code.py'
resa2 = imp.load_source('useful_resa2_code', r2_func_path)

engine, conn = resa2.connect_to_resa2()

# RA database

This notebook documents an initial look at the RADB following KET's e-mails received 16/03/2017 at 09.51 and 23/03/2017 at 14.25. The basic aim is to modify the database so that "summary effects" can be calculated for different taxonomic levels, rather than just for "species groups", as currently.

`ECOTOX_SUMMARY_EFFECT` shows an example of the output available based on "species groups". We want to create similar views for different levels in the taxonomic tree, as defined in the NCBI tables (see KET's e-mails for details). The starting point for this is the `ECOTOX_EXTRACT1_M` view, which contains the basic data, including the Latin name for each organism. The information in this view is then summarised into two more views (`ECOTOX_SUMMARY_EFFECT_PRIMARY` and `ECOTOX_SUMMARY_EFFECT_REVIEW`), and these are subsequently combined into `ECOTOX_SUMMARY_EFFECT`, which is what we're interested in here. 

This notebook aims to:

 1. Get a brief overview of the NCBI data and how it can be linked to `ECOTOX_EXTRACT1_M` <br><br>
 
 2. Understand how we can use the NCBI data to permit aggregation by any taxonomic rank
 
## 1. Example data from `ECOTOX_EXTRACT1_M`

In [3]:
# Get the first 100 rows from ECOTOX_EXTRACT1_M
sql = ("SELECT * FROM risk_assessment.ecotox_extract1_m "
       "WHERE rownum <= 100")
df = pd.read_sql_query(sql, engine)

df.head()

,result_id,test_id,chemical_id,chemical_selection,cas_number_dash,chemical_name,organism_habitat,ecotox_group,species_group,common_name,...,significance_code,significance_type,significance_level_mean_op,significance_level_mean,reference_number,author,title,source,publication_year,invalid
0,2150084,2082355,350,Solution Organics,121-75-5,Malathion,Water,"Fish,Standard Test Species",Fish,Channel Catfish,...,ANOSIG,P,<,0.05,822,"Areechon,N.",Acute and Subchronic Toxicity of Malathion in ...,"Ph.D.Thesis, Auburn University, Auburn, AL:138 p.",1987,None
1,2150011,2082355,350,Solution Organics,121-75-5,Malathion,Water,"Fish,Standard Test Species",Fish,Channel Catfish,...,NOSIG,P,<,0.05,822,"Areechon,N.",Acute and Subchronic Toxicity of Malathion in ...,"Ph.D.Thesis, Auburn University, Auburn, AL:138 p.",1987,None
2,2150013,2082355,350,Solution Organics,121-75-5,Malathion,Water,"Fish,Standard Test Species",Fish,Channel Catfish,...,ANOSIG,P,<,0.05,822,"Areechon,N.",Acute and Subchronic Toxicity of Malathion in ...,"Ph.D.Thesis, Auburn University, Auburn, AL:138 p.",1987,None
3,2150032,2082355,350,Solution Organics,121-75-5,Malathion,Water,"Fish,Standard Test Species",Fish,Channel Catfish,...,NOSIG,P,<,0.05,822,"Areechon,N.",Acute and Subchronic Toxicity of Malathion in ...,"Ph.D.Thesis, Auburn University, Auburn, AL:138 p.",1987,None
4,2150085,2082355,350,Solution Organics,121-75-5,Malathion,Water,"Fish,Standard Test Species",Fish,Channel Catfish,...,ANOSIG,P,<,0.05,822,"Areechon,N.",Acute and Subchronic Toxicity of Malathion in ...,"Ph.D.Thesis, Auburn University, Auburn, AL:138 p.",1987,None


## 2. NCBI dataset

It looks as though there are two key tables: `NCBI_NAMES` and `NCBI_NODES`. The former includes the organism names, while the latter has the taxonomic ranks. It's not immediately obvious to me how to link this information to `ECOTOX_EXTRACT1_M`, but perhaps matching `LATIN_NAME` in the RADB to `NAME_TXT` in the NCBI database is a good place to start?

In [4]:
# First, get all unique latin names from `ECOTOX_EXTRACT1_M`
sql = ("SELECT UNIQUE(latin_name) "
       "FROM risk_assessment.ecotox_extract1_m")
lat_df = pd.read_sql_query(sql, engine)

print 'There are %s unique Latin names in the RADB.' % len(lat_df)

# Try matching these based on NCBI text
sql = ("SELECT * FROM risk_assessment.ncbi_names "
       "WHERE name_txt IN ("
       "SELECT UNIQUE(latin_name) "
       "FROM risk_assessment.ecotox_extract1_m)")
ncbi_df = pd.read_sql_query(sql, engine)

print 'Of these, %s can be matched exactly in the NCBI database.' % len(ncbi_df)

There are 3781 unique Latin names in the RADB.
Of these, 2701 can be matched exactly in the NCBI database.


So, it looks as though we can link the ECOTOX data to the NCBI data based on Latin names in about 70% of cases. This obviously needs improving - **check with KET for options**.

Of these 2701 matches, how many of the 29 NCBI ranks are currently included in the RADB?

In [5]:
# Get matching NCBI ranks
sql = ("SELECT UNIQUE(rank) "
       "FROM risk_assessment.ncbi_nodes "
       "WHERE tax_id IN ("
       "SELECT tax_id FROM risk_assessment.ncbi_names "
       "WHERE name_txt IN ("
       "SELECT UNIQUE(latin_name) "
       "FROM risk_assessment.ecotox_extract1_m))")
rank_df = pd.read_sql_query(sql, engine)

rank_df

,rank
0,suborder
1,subspecies
2,subphylum
3,class
4,kingdom
5,superkingdom
6,genus
7,family
8,phylum
9,no rank


So the databse currently includes entries for 15 of the 29 possible rankings.

For each organism, I need to be able to get (i) its NCBI rank and (ii) the name of every higher taxonomic level that the organism belongs to. The `NCBI_NODES` table does contain this information, but the table structure is recursive: each row has a "parent", which defines the heirarchy. It would be more useful to create a flattened version, with the ranks/taxonomic levels as columns and a row for each `tax_id`, something like this:

| tax_id | rank 1 | rank 2 | rank 3 | rank 4 |
|:------:|:------:|:------:|:------:|:------:|
|    1   |        |        |    1   |    3   |
|    2   |    1   |        |    5   |    7   |

With this structure, it should be straightforward to get taxonomic data for each organism from any level of the taxonomic tree. Generating this table looks a bit tricky, but Oracle has something called "[recursive/heirarchical queries](https://oracle-base.com/articles/misc/hierarchical-queries)", which may be useful in this context. I've never used them before, though, so I'll need to investigate further when I get back in April.

Summary so far:

 * Check with KET regarding linking ECOTOX data to the NCBI codes. Are there better options than linking `LATIN_NAME` to `NAME_TXT`? <br><br>
 
 * `NCBI_NODES` needs restructuring to flatten the recursive data into something we can use for taxonomic aggregation

**Update 05/04/2017:** Returning to this after a week away. Restructuring the NCBI table is tricky, especially because the table is quite large. I posted a question on Stackoverflow [here](http://stackoverflow.com/questions/43212421/flatten-recursive-taxonomic-table-oracle?noredirect=1#comment73499002_43212421) but have had no responses so far. As time is tight, I've decided to write my own code in Python to acomplish the restructuring, as I don't have time to dig into the details of recursive SQL queries at present. The main issue is that my version is likely to be slow, so some optimisation may be necessary.

The first step is to read the entire `NCBI_NODES` table into memory and replace the `rank` text column with an integer column using `rank_ids` running from 0 to 28 (see the Word document received from KET 23/03/2017 at 14.25 for details). I can then extract just the three columns of interest - `tax_id`, `parent_tax_id` and `rank_id`. 

This table will be quite large: if all values are stored as 64 bit integers, I should end up with 1560278 rows and four columns (three of values, plus the index). This will occupy approximately

$$\frac{1560278 * 4 * 64}{8.10^6} = 49.93 MB$$

in memory. Although this is pretty big, I should still be able to work with it on my laptop quite easily.

In [6]:
# Read table of NCBI ranks
in_xlsx = r'C:\Data\James_Work\Staff\Knut_Eric_T\RA_Database\ncbi_ranks.xlsx'
rank_df = pd.read_excel(in_xlsx, sheetname='Sheet1')

# Read entire NCBI_NODES table
sql = ("SELECT tax_id, parent_tax_id, rank "
       "FROM risk_assessment.ncbi_nodes")
ncbi_df = pd.read_sql_query(sql, engine)

# Join rank ids
ncbi_df = ncbi_df.merge(rank_df, how='left',
                        on='rank')
del ncbi_df['rank']

print 'Total number of records:', len(ncbi_df)
print 'Size of data frame in memory: %.2f MB' % (ncbi_df.memory_usage().sum()/1.E6)

ncbi_df.head()

Total number of records: 1560278
Size of data frame in memory: 49.93 MB


,tax_id,parent_tax_id,rank_id
0,211,209,4
1,212,209,4
2,213,209,4
3,214,209,4
4,215,209,4


It's reassuring to see that the size of the data frame in memory is exactly as predicted. The hard bit is how to flatten the recursive relationship. Note also that the results array will be pretty large: 156028 records and 31 columns (including the index) should occupy about 387 MB. 

The code below performs the flattening using Pandas. Because I'm concerned about performance, I've wrapped everything in a function to give me some options for optimisation and benchmarking later on.

In [7]:
%%time
def flatten_pandas(ncbi_df):
    """ Use Pandas to flatten the recursive NCBI table.
    """
    # Pre-allocate results array in memory
    data = np.zeros((len(ncbi_df), 30))*np.nan
    tax_df = pd.DataFrame(data=data, columns=['tax_id']+range(1,29)+[0])

    # Loop over tax_id
    for idx, row in ncbi_df[:1000].iterrows():
        # Add tax_id to output
        tax_df.ix[idx, 'tax_id'] = row.tax_id

        # Add tax_id to the appropriate level
        tax_df.ix[idx, row.rank_id] = row.tax_id

        # Walk tree
        par = row.parent_tax_id    
        while par != 1: # parent = 1 is the top level of the tree
            # Get the rank of the parent
            par_rank = ncbi_df[ncbi_df['tax_id']==par].rank_id.iloc[0]

            # Enter the parent at the appropriate rank
            tax_df.ix[idx, par_rank] = par

            # Get the next parent
            par = ncbi_df[ncbi_df['tax_id']==par].parent_tax_id.iloc[0]

    return tax_df

# Flatten data
tax_df = flatten_pandas(ncbi_df)
print tax_df.head()

   tax_id   1   2   3      4   5   6   7      8   9    ...     20       21  \
0   211.0 NaN NaN NaN  211.0 NaN NaN NaN  209.0 NaN    ...    NaN  29547.0   
1   212.0 NaN NaN NaN  212.0 NaN NaN NaN  209.0 NaN    ...    NaN  29547.0   
2   213.0 NaN NaN NaN  213.0 NaN NaN NaN  209.0 NaN    ...    NaN  29547.0   
3   214.0 NaN NaN NaN  214.0 NaN NaN NaN  209.0 NaN    ...    NaN  29547.0   
4   215.0 NaN NaN NaN  215.0 NaN NaN NaN  209.0 NaN    ...    NaN  29547.0   

   22       23      24  25  26  27   28         0  
0 NaN  68525.0  1224.0 NaN NaN NaN  2.0  131567.0  
1 NaN  68525.0  1224.0 NaN NaN NaN  2.0  131567.0  
2 NaN  68525.0  1224.0 NaN NaN NaN  2.0  131567.0  
3 NaN  68525.0  1224.0 NaN NaN NaN  2.0  131567.0  
4 NaN  68525.0  1224.0 NaN NaN NaN  2.0  131567.0  

[5 rows x 30 columns]
Wall time: 8min 3s


This code works but, as I feared, it's very inefficient. The test example above takes 7.5 minutes to process the first 1000 rows, which means the full table will take around $1500*7.5$ minutes, which is more than a week! Clearly this isn't good enough, so time for some optimisation...

Using Pandas makes for much cleaner/easier code, but cutting away this convenience and dropping down to more basic Numpy should be faster. As a first step, I'll convert `ncbi_df` to a `dict`. This will take up more memory, but should hopefully provide fast data access based on `tax_id`.

In [8]:
# First convert ncbi_df to a dict
ncbi_df.index = ncbi_df['tax_id']
del ncbi_df['tax_id']

ncbi_dict = ncbi_df.to_dict(orient='index')
print 'Size of ncbi_dict: %.2f MB.' % (sys.getsizeof(ncbi_dict)/1.E6)

Size of ncbi_dict: 100.66 MB.


So the data is now taking up twice as much memory as before, but it's still manageable. The code below removes all the Pandas indexing sophistication and uses basic Numpy instead.

In [9]:
%%time
def flatten_numpy(ncbi_dict):
    """ Use Numpy to flatten the recursive NCBI table.
    """
    # Pre-allocate results array in memory
    data = np.zeros((len(ncbi_dict.keys()), 30))*np.nan

    # Loop over tax_id
    for idx, tax_id in enumerate(ncbi_dict.keys()[:1000]):
        # Add tax_id to output
        data[idx, 0] = tax_id

        # Add tax_id to the appropriate level
        rank_id = ncbi_dict[tax_id]['rank_id']
        if rank_id == 0:
            rank_id = 29
        data[idx, rank_id] = tax_id

        # Walk tree
        par = ncbi_dict[tax_id]['parent_tax_id']  
        while par != 1: # parent = 1 is the top level of the tree
            # Get the rank of the parent
            par_rank = ncbi_dict[par]['rank_id']

            # Enter the parent at the appropriate rank
            if par_rank == 0:
                par_rank = 29
            data[idx, par_rank] = par

            # Get the next parent
            par = ncbi_dict[par]['parent_tax_id']

    # Build df
    tax_df = pd.DataFrame(data=data, columns=['tax_id']+range(1,29)+[0])
    
    return tax_df

# Flatten data
tax_df = flatten_numpy(ncbi_dict)
print tax_df.head()

   tax_id   1   2   3    4   5   6   7        8   9    ...     20       21  \
0     1.0 NaN NaN NaN  NaN NaN NaN NaN      NaN NaN    ...    NaN      NaN   
1     2.0 NaN NaN NaN  NaN NaN NaN NaN      NaN NaN    ...    NaN      NaN   
2     6.0 NaN NaN NaN  NaN NaN NaN NaN      6.0 NaN    ...    NaN  28211.0   
3     7.0 NaN NaN NaN  7.0 NaN NaN NaN      6.0 NaN    ...    NaN  28211.0   
4     9.0 NaN NaN NaN  9.0 NaN NaN NaN  32199.0 NaN    ...    NaN   1236.0   

   22  23      24  25  26  27   28         0  
0 NaN NaN     NaN NaN NaN NaN  NaN       1.0  
1 NaN NaN     NaN NaN NaN NaN  2.0  131567.0  
2 NaN NaN  1224.0 NaN NaN NaN  2.0  131567.0  
3 NaN NaN  1224.0 NaN NaN NaN  2.0  131567.0  
4 NaN NaN  1224.0 NaN NaN NaN  2.0  131567.0  

[5 rows x 30 columns]
Wall time: 369 ms


Wow - this is a remarkable speedup! For the first 1000 rows we're down from 7.5 minutes to 0.4 seconds, a factor of more than 1000! How about the full dataset?

In [10]:
%%time
def flatten_numpy(ncbi_dict):
    """ Use Numpy to flatten the recursive NCBI table.
    """
    # Pre-allocate results array in memory
    data = np.zeros((len(ncbi_dict.keys()), 30))*np.nan

    # Loop over tax_id
    for idx, tax_id in enumerate(ncbi_dict.keys()):
        # Add tax_id to output
        data[idx, 0] = tax_id

        # Add tax_id to the appropriate level
        rank_id = ncbi_dict[tax_id]['rank_id']
        if rank_id == 0:
            rank_id = 29
        data[idx, rank_id] = tax_id

        # Walk tree
        par = ncbi_dict[tax_id]['parent_tax_id']  
        while par != 1: # parent = 1 is the top level of the tree
            # Get the rank of the parent
            par_rank = ncbi_dict[par]['rank_id']

            # Enter the parent at the appropriate rank
            if par_rank == 0:
                par_rank = 29
            data[idx, par_rank] = par

            # Get the next parent
            par = ncbi_dict[par]['parent_tax_id']

    # Build df
    tax_df = pd.DataFrame(data=data, columns=['tax_id']+range(1,29)+[0])
    
    return tax_df

# Flatten data
tax_df = flatten_numpy(ncbi_dict)
print tax_df.head()

   tax_id   1   2   3    4   5   6   7        8   9    ...     20       21  \
0     1.0 NaN NaN NaN  NaN NaN NaN NaN      NaN NaN    ...    NaN      NaN   
1     2.0 NaN NaN NaN  NaN NaN NaN NaN      NaN NaN    ...    NaN      NaN   
2     6.0 NaN NaN NaN  NaN NaN NaN NaN      6.0 NaN    ...    NaN  28211.0   
3     7.0 NaN NaN NaN  7.0 NaN NaN NaN      6.0 NaN    ...    NaN  28211.0   
4     9.0 NaN NaN NaN  9.0 NaN NaN NaN  32199.0 NaN    ...    NaN   1236.0   

   22  23      24  25  26  27   28         0  
0 NaN NaN     NaN NaN NaN NaN  NaN       1.0  
1 NaN NaN     NaN NaN NaN NaN  2.0  131567.0  
2 NaN NaN  1224.0 NaN NaN NaN  2.0  131567.0  
3 NaN NaN  1224.0 NaN NaN NaN  2.0  131567.0  
4 NaN NaN  1224.0 NaN NaN NaN  2.0  131567.0  

[5 rows x 30 columns]
Wall time: 24.6 s


25 seconds for the full thing is pretty good, and certainly good enough for this application. However, for future reference, note that additional performance increases are possible using [Numba](http://numba.pydata.org/), simply use the following:

    from numba import jit
    
and then add the `@jit` decorator to the function call:

    @jit
    def flatten_numpy(ncbi_dict):
        # code here

This automatically re-writes the function C/++ and gives further speed improvements. If you need to go even faster, these operations could be parallelised by splitting `ncbi_df` into chunks and using something like this:

    from multiprocessing import Pool

    @jit # Optional Numba optimisation
    def flatten_numpy(ncbi_df):
        # code here

    if __name__ == '__main__':
        p = Pool(8) # Number of cores to use
        p.map(flatten_numpy, [ncbi_df1, ncbi_df2, ..., ncbi_df8])
        
The final step is to write the output back to the RA database for inspection.

In [12]:
# Write results to RAdb
tax_df.to_sql('ncbi_nodes_flat', 
              schema='risk_assessment',
              con=engine, index=False)

This seems to have worked, and some basic manual checking using [this website](https://www.ncbi.nlm.nih.gov/Taxonomy/TaxIdentifier/tax_identifier.cgi) gives compatible results. Note, however, that in some cases the `NCBI_NODES` table has multiple parents with rank 0. As an example:

    tax_id 2499 (rank 4) > tax_id 36549 (rank 0) > tax_id 28384 (rank 0)
    
My code only allows a single `tax_id` for each rank, and in situations like this it is the highest taxonomic level that gets assigned i.e. my code returns the tree:

    tax_id 2499 (rank 4) > tax_id 28384 (rank 0)
    
(missing out 36549), because 36549 and 28384 both have rank 0, but 28384 is at a higher taxonomic level. This seems reasonable to me, but **check with KET**.

## 3. Updating views

The flattened NCBI data provides full taxonomic information for each `tax_id`. In principle, this should make it possible to perform aggregation at any taxonomic level. 

**NB:** Back in November 2016, Knut Erik asked me to modify some existing views in the database, such that grouping took place based on the `LATIN_NAME` column rather than the `SPECIES_GROUP`. In the end, I created a new view called `ECOTOX_SUMMARY_EFFECT_SPECIES` (see e-mails around 14/11/2016 for details).

Using the newly formatted NCBI data, it should be possible to generate a similar table by grouping based on `NCBI_RANK=4`, which corresponds to species. We can then check to see how aggregation via NCBI ranks compares to aggregation using Latin names, which should provide a useful test. To explore this, I've used the following workflow in the database:

 1. Created two new views, `ECOTOX_EXTRACT1_M_NCBI` and `ECOTOX_LIKE_EXTRACT1_NCBI`. These are identical to `ECOTOX_EXTRACT1_M` and `ECOTOX_LIKE_EXTRACT1`, except I've joined-in the NCBI ranks by matching NCBI `NAME_TXT` to RAdb `LATIN_NAME`. As noted above, not all the names can be matched, so this **needs further checking**. <br><br>
 
 2. Created two new views called `ECOTOX_SUM_EFF_PRI_SPEC` and `ECOTOX_SUM_EFF_REV_SPEC`, which are the same as `ECOTOX_SUMMARY_EFFECT_PRIMARY` and `ECOTOX_SUMMARY_EFFECT_REVIEW`, except the grouping is performed on `NCBI_RANK=4` (species) instead of on `LATIN_NAME` or `SPECIES_GROUP` as we have done previously. <br><br>
 
 3. Created a new view called `ECOTOX_SUM_EFF_SPEC`, which combines the results from `ECOTOX_SUM_EFF_PRI_SPEC` and `ECOTOX_SUM_EFF_REV_SPEC` (exactly analogous to the way `ECOTOX_SUMMARY_EFFECT` combines output from `ECOTOX_SUMMARY_EFFECT_PRIMARY` and `ECOTOX_SUMMARY_EFFECT_REVIEW`).
 
In summary, `ECOTOX_SUM_EFF_SPEC` is based on the NCBI data aggregated at `RANK=4`, whereas the table I created back in November (called `ECOTOX_SUMMARY_EFFECT_SPECIES`) is based on aggregation using the RAdb `LATIN_NAME`. These two tables are similar, but not the same, so **they need checking carefully to make sure the values based on the NCBI data look reasonable**.

The following issues need further investigation:

 * Not all Latin names in our database have exact matches in the NCBI database.  <br><br>
 
 * In a few cases, the Latin name specified in the RAdb matches more than one tax_id in NCBI_NAMES. This results in some duplication i.e. the same Latin name gets assigned to more than one level in the taxonomic tree. This may lead to contradictory results.